# Evolver Loop 1 - LB Feedback Analysis

## LB Score: 70.6473 (matches CV exactly)

This confirms our scoring is calibrated correctly. Now we need to:
1. Compile bbox3 from source (pre-compiled binary has GLIBC issues)
2. Run optimization to improve the score
3. Target: 68.919 (need ~1.73 point improvement)

In [ ]:
# Extract bbox3.cpp source from the jazivxt kernel
import json

with open('/home/code/research/kernels/jazivxt_why-not/why-not.ipynb', 'r') as f:
    nb = json.load(f)

# Find the cell with bbox3.cpp
for cell in nb['cells']:
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source']) if isinstance(cell['source'], list) else cell['source']
        if '%%writefile bbox3.cpp' in source:
            # Extract the C++ code (skip the %%writefile line)
            lines = source.split('\n')
            cpp_code = '\n'.join(lines[1:])  # Skip first line
            print(f"Found bbox3.cpp source: {len(cpp_code)} characters")
            print("First 500 chars:")
            print(cpp_code[:500])
            break

In [ ]:
# Save the C++ source
with open('/home/code/exploration/bbox3.cpp', 'w') as f:
    f.write(cpp_code)

print("Saved bbox3.cpp")
print(f"File size: {len(cpp_code)} bytes")

In [ ]:
# Compile bbox3
import subprocess

result = subprocess.run(
    ['g++', '-O3', '-fopenmp', '-o', '/home/code/exploration/bbox3', '/home/code/exploration/bbox3.cpp'],
    capture_output=True,
    text=True
)

print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)
print("Return code:", result.returncode)

In [ ]:
# Test the compiled binary
import subprocess
import os

os.chdir('/home/code/exploration')

# Copy submission to working directory
import shutil
shutil.copy('/home/code/exploration/datasets/submission.csv', '/home/code/exploration/submission.csv')

# Run bbox3 with a short test
result = subprocess.run(
    ['./bbox3', '-n', '100', '-r', '30'],
    capture_output=True,
    text=True,
    timeout=60
)

print("STDOUT:")
print(result.stdout[:2000] if result.stdout else "(empty)")
print("\nSTDERR:")
print(result.stderr[:500] if result.stderr else "(empty)")
print("\nReturn code:", result.returncode)

In [ ]:
# Check the score after optimization
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

# Tree shape coordinates
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, deg):
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = rotate(poly, deg, origin=(0, 0))
    poly = translate(poly, x, y)
    return poly

def calculate_bounding_box_side(trees):
    all_x = []
    all_y = []
    for x, y, deg in trees:
        poly = create_tree_polygon(x, y, deg)
        bounds = poly.bounds
        all_x.extend([bounds[0], bounds[2]])
        all_y.extend([bounds[1], bounds[3]])
    width = max(all_x) - min(all_x)
    height = max(all_y) - min(all_y)
    return max(width, height)

def parse_submission(filepath):
    df = pd.read_csv(filepath)
    def parse_val(v):
        if isinstance(v, str) and v.startswith('s'):
            return float(v[1:])
        return float(v)
    df['x_val'] = df['x'].apply(parse_val)
    df['y_val'] = df['y'].apply(parse_val)
    df['deg_val'] = df['deg'].apply(parse_val)
    df['N'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    result = {}
    for n, group in df.groupby('N'):
        trees = list(zip(group['x_val'], group['y_val'], group['deg_val']))
        result[n] = trees
    return result

def calculate_score(side_lengths):
    return sum(s**2 / n for n, s in side_lengths.items())

# Calculate score for the optimized submission
trees_by_n = parse_submission('/home/code/exploration/submission.csv')
side_lengths = {}
for n in range(1, 201):
    if n in trees_by_n:
        side = calculate_bounding_box_side(trees_by_n[n])
        side_lengths[n] = side

total_score = calculate_score(side_lengths)
print(f"Score after bbox3 optimization: {total_score:.6f}")
print(f"Original baseline: 70.647327")
print(f"Improvement: {70.647327 - total_score:.6f}")